# Building a Bigram Language Model from Scratch
---

This is an extended version of Andrej Karpathy's notebook in addition to his [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on bigram language models.

Adapted by: 

Prof. Dr.-Ing. Antje Muntzinger, University of Applied Sciences Stuttgart

antje.muntzinger@hft-stuttgart.de

---


We'll construct a bigram language model from scratch. 
The model will be trained on a text file containing names and will be able to generate new names based on what it has learned. So if you are expecting a baby and looking for some extraordinary new name, you might get some inspiration here :)

# Table of Contents

[1. Loading the Data](#1.-loading-the-data)

[2. Preprocessing the Data](#2.-preprocessing-the-data)

[3. Bigram Counts as PyTorch Tensor](#3.-Bigram-Counts-as-PyTorch-Tensor)

[4. Sampling New Characters](#4.-sampling-new-characters)

[5. Broadcasting](#5.-broadcasting)

[6. Generating New Names](#6.-generating-new-names)

[7. Evaluating the Model](#7.-evaluating-the-model)

[8. The Neural Network Approach](#8.-the-neural-network-approach)

[9. The First Neuron](#9.-The-First-Neuron)

[10. Creating 27 Neurons](#10.-Creating-27-Neurons)

[11. Summary (so far)](#11.-Summary-(so-far))

[12. Optimization](#12.-Optimization)

[13. Optimizing over the Whole Dataset](#13.-Optimizing-over-the-Whole-Dataset)

[14. Sampling from the Neural Net](#14.-Sampling-from-the-Neural-Net)

[15. Conclusion](#15.-Conclusion)

## 1. Loading and Preprocessing the Data

First, we read the names from the text file and save them in a list.

In [3]:
words = open('names.txt', 'r').read().splitlines()

**TODO:** 1) Print the first 10 names. Find out the number of names contained in the dataset as well as the shortest and longest name. **(2 points)**

In [4]:
# YOUR CODE GOES HERE
print("First 10 names in the dataset: ", words[:10])
print("Number of names contained in the dataset: ", len(words))
print("Longest name in the dataset: ", max(words, key=len))
print("Shortest name in the dataset: ",min(words, key=len))

Note that in one example name like "isabella", a lot of information is contained. For example:
- "i" is likely to be the first character of a name
- "s" is likely to follow after an "i"
- "a" is likely to follow after "is"
- ...
- after "isabella", the name is likely to end.

Here we are going to create a **bigram** language model, which means we only use the single previous character to predict the next. For example, we use "s" (not "is") to predict "a" in "isabella" and forget that we have a lot more information.

## 2. Preprocessing the Data

First, we introduce two special characters, `<S>` for "start" and `<E>` for "end". Then we zip two consecutive characters and print them: 

In [ ]:
b = {} # dictionary to store bigram counts
for w in words: # print bigrams
    chs = ['<S>'] + list(w) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]): # zip(['a', 'b', 'c'], ['b', 'c', 'd']) -> [('a', 'b'), ('b', 'c'), ('c', 'd')]
        bigram = (ch1, ch2)
        # count how many times bigram appears
        b[bigram] = b.get(bigram, 0) + 1 # b.get(bigram, 0) returns b[bigram] if bigram is in b, otherwise 0
        print(ch1, ch2, b[bigram])

We see that already in the first three names, the bigram `a <E>` occurs three times - which makes sense because many names end with an "a".

In [ ]:
# print all bigrams
b

In [ ]:
# sort by the count (by default it sorts by the first element of the tuple, but we want the second element)
sorted(b.items(), key = lambda kv: kv[1], reverse=True) # kv is a tuple, kv[0] is the key, kv[1] is the value

**TODO:** 2) What is the most common bigram in the dataset? What is the least common bigram? **(2 points)**


In [8]:
# YOUR CODE GOES HERE 
print("most common bigram:", max(b, key=b.get))
print("least common bigram:", min(b, key=b.get))

## 3. Bigram Counts as PyTorch Tensor

Next, we store the bigrams in a matrix instead of a python dictionary: The rows are going to be the first character of the bigram and the columns are going to be the second character. Each entry in this matrix will tell us how often that first character is followed by the second character in the dataset.

To create this matrix (also called **2D array** or **2D tensor**), we use PyTorch, which allows us to create multi-dimensional arrays and manipulate them very efficiently. We will also use a special character `.` for word boundaries instead of `<A>` and `<E>` because we don't need to distinguish between these two.

In [9]:
import torch

We get all characters and map them to integers in a mapping called "s to i":

In [10]:
chars = sorted(list(set(''.join(words)))) # get all unique characters in words
stoi = {s:i+1 for i,s in enumerate(chars)} # create a dictionary mapping characters to integers
stoi['.'] = 0 # add a special character for word boundaries instead of <A> and <E>

We also create the inverse mapping "i to s" that assigns a character to each integer. We can use `items()` to get a list of a dictionary's tuples:

In [ ]:
print(stoi)
print(stoi.items())

In [10]:
itos = {i:s for s,i in stoi.items()} # reverse mapping from integers to characters

**TODO:** 3a) We now have a vocabulary of 27 characters. 
Instanciate a torch tensor called `N` of size 27x27 with zero values of dtype int32. Check the PyTorch documentation to see how to create a tensor if needed. **(1 point)**

In [11]:
# YOUR CODE GOES HERE 
N = torch.zeros((len(stoi), len(stoi)), dtype=torch.int32)

Let's fill the matrix `N` with the counts of the bigrams:

In [12]:
# store the counts of bigrams in the matrix
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [13]:
# print the matrix
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(16,16))
plt.imshow(N, cmap='Blues')
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j] # character string of bigram, e.g. 'ab'
        plt.text(j, i, chstr, ha="center", va="bottom", color='gray') # upper text: bigram
        plt.text(j, i, N[i, j].item(), ha="center", va="top", color='gray') # lower text: count
plt.axis('off')

**TODO:** 3b) Why do we use `.item()` in the code cell above? Check out the PyTorch documentation if needed! **(1 point)**

**ANSWER:** YOUR ANSWER GOES HERE

The .item() method is used to extract the value of a single element from a PyTorch tensor and convert it from the tensor N[i, j] (which is a 0-dimensional tensor) to an integer before being passed to plt.text(). This is necessary because plt.text() expects standard Python data types, not PyTorch tensor objects.

Without .item(), PyTorch would return a tensor object (like tensor(5)), which might cause issues or unexpected behavior when trying to display the value using matplotlib.

**TODO:** 3c) Assume our first character is an 'f'. Looking at the matrix plot above, which is the most likely next character? **(1 point)**

In [ ]:
# YOUR ANSWER GOES HERE
print(itos[torch.argmax(N[stoi['f']]).item()])

In [ ]:
print(N[1,2])
print(N[1,2].item())

Here is a visual summary so far - this is how we can get the most likely next character for input 'e' by plucking out the 6th row:

<img src="img/bigram1.jpg" width="500">

## 4. Sampling New Characters

Now we want to sample new characters. We start with the first character `.`, 
and then sample the next character based on the count of bigrams.
We have the raw counts stored in the matrix N, but we need to convert them to 
probabilities in order to sample from the distribution.

**TODO:** 4a) Pluck out the first row of the matrix, convert the entries to floats, and normalize them so that the row sums to 1. **(3 points)** 

**HINT:** You can simply cast a PyTorch tensor to float using `.float()`, see for example here: https://www.datascienceweekly.org/tutorials/pytorch-change-tensor-type-cast-a-pytorch-tensor-to-another-type

In [15]:
# YOUR CODE GOES HERE 
p = N[0].float() # convert first row of N to float
p /= p.sum() # normalize values
print(p)


**TODO:** 4b) How can you interpret this row now? **(1 point)**

**ANSWER:** YOUR ANSWER GOES HERE

"All values are between 0 and 1 and can be interpreted as a nominal probability"

To sample from this distribution, we use `torch.multinomial`, which samples from the multinomial probability distribution (in simple words: "you give me probabilities and I will give you integers sampled according to the probability distribution").

In [16]:
g = torch.Generator().manual_seed(2147483647) # makes the random numbers reproducible
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item() # sample from the distribution
# replacement=True means that we are sampling with replacement (i.e. we can sample the same index multiple times)
print("sampled index:", ix) # print the sampled index
itos[ix] # convert the sampled index to a character

**TODO:** 4c) Sample 100 characters according to your probability distribution for word beginnings. Print the sampled indices and characters. Can you relate the output to your probability distribution? Did you expect this output, or can you see unexpected output characters? **(2 points)**

In [17]:
# YOUR CODE GOES HERE 
g = torch.Generator().manual_seed(2147483647)
ix_100 = torch.multinomial(first_row, num_samples=100, replacement=True, generator=g) # use torch.multinominal with batch size of 100
for i in ix_100:
    print(i.item(),":", itos[i.item()])

**ANSWER:** YOUR ANSWER GOES HERE

Yes, i can relate the output to probability distribution. I expect this output because the letters with highest probabilities are : a,b,c,d,e,j,k,l,m,n and they are found in output more than others. Also the probability of '..' was 0 and i have never seen it on the output.

This is the updated visualization including normalization for probabilities as outputs:

<img src="img/bigram2.jpg" width="600">

---

## 5. Broadcasting
We store all probabilities in a matrix `P`, so that each row is normalized to 1 and contains the probabilities for the next character:

In [18]:
P = (N+1).float() 
P /= P.sum(1, keepdims=True) # normalize the rows of P (we use keepdims=True to keep the dimensions of P, see below)
print(P)

**TODO:** 5) Can you think of a reason why we calculate the probabilities based on `N+1` instead of `N`? **(1 point)**

**HINT:** What happens for `N=0`?

**ANSWER:** YOUR ANSWER GOES HERE

It is a so called Laplace-smoothing which makes sure that no entry is 0. This is needed not have any 0% probability for the entries without occurrence in the training data.

We need `keepdims=True` in order to sum each line, the result should be a 27x1 column vector containing the row sums - otherwise all entries get collapsed to a 1D instead of 2D array: 

In [19]:
print(P.sum(1, keepdims=True).shape) # shape collapes to 27x1 due to summation along axis 1 (column vector)
print(P.sum(1, keepdims=False).shape) # 1D tensor of size 27

But why does the division above `P /= P.sum(1, keepdims=True)` even work? We divide the 27x27 matrix `P` by a 27x1 vector. This only works because PyTorch automatically applies **broadcasting**, a type of tensor manipulation: From https://pytorch.org/docs/stable/notes/broadcasting.html we get the information that two tensors are **broadcastable** if the following rules hold:

    Each tensor has at least one dimension.

    When iterating over the dimension sizes, starting at the trailing dimension, the dimension sizes must either be equal, one of them is 1, or one of them does not exist.

If two tensors x, y are “broadcastable”, the resulting tensor size is calculated as follows:

    If the number of dimensions of x and y are not equal, prepend 1 to the dimensions of the tensor with fewer dimensions to make them equal length.

    Then, for each dimension size, the resulting dimension size is the max of the sizes of x and y along that dimension.
    
In our case, broadcasting takes the 27x1 column vector and copies it 27 times to get a 27x27 matrix, then it makes an element-wise division. We can check the result:

In [20]:
# check that the first row sums to 1
print(P[0].sum())

**TODO (optional):** What happens if you remove `keepdims=True`? Will the division still result in the same matrix - why or why not? 

 **HINT:** Don't forget to revert your changes for later cells to work right - or copy the code cell and use different variable names.


**ANSWER:** YOUR ANSWER GOES HERE

If we remove it, it will return 1D array(each element is sum of one row),then when we divide the tensor(27,27) with sum(27). As it didnt protect the dimension features, the division would be wrong(maybe column wise). In the code below, as there is a mistake in calculation, sum of first row is not 1 :

In [21]:
# YOUR CODE GOES HERE 

P_v2 = (N+1).float()
P_v2 /= P_v2.sum(1)
print("If it would normalized correctly sum supposed to be 1 but : ",P_v2[0].sum())

In [22]:
# YOUR CODE GOES HERE 
# we are actually normalizing the columns of P, not the rows:
print('row sums:', P1[0].sum(), P2[0].sum())
print('column sums:', P1[:,0].sum(), P2[:,0].sum())

## 6. Generating New Names

With this, we can sample new names by starting with the start character `.` and sampling the next character, then the next next character and so on, until we sample the end character `.`:

In [23]:
g = torch.Generator().manual_seed(2147483647)

for i in range(20): # generate 20 names
  
    out = []
    ix = 0 # start with the start-of-word character
    while True:
        p = P[ix] # get the probabilities of the next character by plucking the row corresponding to the current character
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item() # sample the next character (=column index)
        out.append(itos[ix]) # convert the sampled index to a character and append it to the output
        if ix == 0: # if we sampled the end-of-word character: break
            break
    print(''.join(out)) # print the generated name

This doesn't look right... but it is! The bigram language model is simply not  powerful enough to create more reasonable names. For example, it generated a name "a" twice, but it doesn't know that "a" is the first character here. All it knows is that "a" is a character in the name and how likely it is that the name ends here, without looking at previous characters.

**TODO:** 6) In the code cell above, experiment with replacing `p` with the uniform distribution, making everything equally likely. Can you see that the bigram model works better than pure chance? **(2 points)**

**HINT:** Don't forget to revert your changes for later cells to work right - or copy the code cell and use different variable names.

In [24]:

g = torch.Generator().manual_seed(2147483647)

for i in range(20):  
    out = []
    ix = 0  # Start mit dem Startzeichen '.'
    
    while True:
        # Ersetzen von `p` mit einer Gleichverteilung
        p = torch.ones_like(P[ix])  # Erstellen eines Vektors gleicher Wahrscheinlichkeiten
        p /= p.sum()  # Normieren, sodass sich die Summe auf 1 beläuft (Gleichverteilung)
        
        # Ziehe das nächste Zeichen
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])  # Füge das Zeichen zur Ausgabe hinzu
        
        # Schleife abbrechen, wenn das Endzeichen `.` gesampelt wird
        if ix == 0:
            break
    
    print(''.join(out))  # Gib den generierten Namen aus


## 7. Evaluating the Model

We want to evaluate the quality of the model using a single number, the **training loss**. But which single number to use? Let's print the probabilities of the first bigrams to start with:

In [25]:
for w in words[:3]: # print bigram probabilities for the first 3 words
    chs = ['.'] + list(w) + ['.'] # add start and end of word characters
    for ch1, ch2 in zip(chs, chs[1:]): # iterate over bigrams
        ix1 = stoi[ch1] # get the integer representation of the characters
        ix2 = stoi[ch2] # get the integer representation of the characters
        prob = P[ix1, ix2] # get the probability of the bigram
        print(f'{ch1}{ch2}: {prob:.4f}') # print the bigram and its probability

**TODO:** 7a) What probabilities would we expect for an untrained model? How do you interpret these outputs compared to the untrained model? **(2 points)**

**ANSWER:** YOUR ANSWER GOES HERE

For an untrained model, we would expect all bigram probabilities to be equal. This is because, in an untrained model, we have no prior information about the likelihood of different character transitions.

If there are 27 possible characters (including the start and end markers), each bigram probability in an untrained model would be close to 1/27 ≈ 0.037

So how can we summarize these probabilities in a single number? First of all, we can use the **likelihood**, which is the product of all these probabilities. The likelihood tells us which probability our model assigns to the whole dataset. So we want the likelihood to be as big as possible for a good model (**maximum likelihood estimation**). 

**TODO**: 7b) Can you think of a problem when calculating the likelihood as defined above? **(1 point)**

**HINT**: What happens numerically if we multiply many tiny numbers?

**ANSWER:** YOUR ANSWER GOES HERE

Since the likelihood is the product of probabilities (which are typically small), it can quickly approach very small values, close to zero, as the dataset grows.

Instead of the likelihood, we use the **log likelihood**: Applying the logarithm transforms the likelihood to a value in $[-\infty, 0]$.

In [26]:
# Generate input values between 0 and 1
x = torch.linspace(0.001, 1, 100)  # Avoid log(0) by starting from 0.001

# Compute the logarithm of the input values
y = torch.log(x)

# Create the plot
plt.figure(figsize=(4, 3))
plt.plot(x, y, label='log(x)')
plt.title('Logarithm of Input Values Between 0 and 1')
plt.xlabel('Input Value')
plt.ylabel('Logarithm')
plt.legend()
plt.grid(True)
plt.show()

**TODO:** 7c) Why is maximizing the likelihood equivalent to maximizing the log likelihood? **(1 point)**

**ANSWER:** YOUR ANSWER GOES HERE

Because the logarithm is a monotonically increasing function. This means that if you increase the likelihood, its logarithm also increases, and vice versa.

Instead of maximizing the log likelihood, in practice we minimize the negative log likelihood, because a loss usually is a number in $[0,\infty]$, where small numbers are good. Finally, the negative log likelihood is simply the sum of the single logarithms due to

$$\log(a*b*c) = \log(a) + \log(b) + \log(c)$$

This is great because a single probability close to zero will not cause the whole loss to be tiny any more, we now use addition instead of multiplication!

**TODO:** 7d) Calculate the **negative log likelihood loss**, store it in `nll` and print it. Sometimes, we also use the **average negative log likelihood loss**, so average `nll` by the number of bigrams evaluated and print the average as well. **(3 points)**

**HINT:** You can start by copying the second last code cell above, where we calculated the probabilities of bigrams. Then apply `torch.log` to these probabilities and sum the negative logarithms.

In [27]:
# YOUR CODE GOES HERE 

nll = 0  # initialize negative log likelihood loss
num_bigrams = 0  # counter for the total number of bigrams

for w in words:  # loop over each word in the dataset
    chs = ['.'] + list(w) + ['.']  # add start and end markers to each word
    for ch1, ch2 in zip(chs, chs[1:]):  # iterate through each bigram in the word
        ix1 = stoi[ch1]  # index for first character
        ix2 = stoi[ch2]  # index for second character
        prob = P[ix1, ix2]  # get the probability of the bigram
        nll += -torch.log(prob)  # add the negative log probability to nll
        num_bigrams += 1  # increment the bigram count

# Average negative log likelihood
avg_nll = nll / num_bigrams

# Print results
print("Negative Log Likelihood Loss:", nll.item())
print("Average Negative Log Likelihood Loss:", avg_nll.item())

This summarizes our first bigram model, which simply counts occurences of bigrams and generates new names based on the corresponding probability distribution. The average log likelihood loss of this model is around 2.45.

The following visualization includes the whole bigram model with negative log likelihood loss:

<img src="img/bigram3.jpg" width="800">

---

## 8. The Neural Network Approach

Let's try another approach: a neural network! Our neural network will still be a bigram language model: It receives a single character as an input, processes it using some weights or some parameters w and outputs the probability distribution over the next character. We will evaluate the parameters of the neural net using the negative log likelihood loss, which means comparing the output probability distribution and the label (the identity of the next character). We are going to use gradient-based optimization to tune the parameters of this network to minimize the loss with the goal to better predict the next character. In the end, the result will look quite similar to the intuitive approach counting occurances of bigrams above!

First, let's create the training set for the neural network made of two lists, the inputs and the targets: 

In [28]:
# create the training set of bigrams (x,y)
xs, ys = [], []

for w in words[:1]: # only use the first word for now, which contains 5 examples
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        print(ch1, ch2)
        xs.append(ix1) # input is the first character of the bigram
        ys.append(ix2) # target is the second character of the bigram
    
xs = torch.tensor(xs) # convert the list to a tensor
ys = torch.tensor(ys) # convert the list to a tensor

We print the 5 resulting inputs and targets:

In [29]:
xs

In [30]:
ys

**TODO:** 8) The following code cell applies **one-hot-encoding**. Shortly explain what this is and why it is needed! **(2 points)**

**ANSWER:** YOUR ANSWER GOES HERE

One-hot encoding allows us to convert categorical data (like characters) into a numerical format that can be used as input for neural networks. Neural networks expect numerical input, and one-hot vectors ensure each character has a unique, distinguishable vector representation without introducing any inherent ordering or numerical relationships between characters.

In [31]:
# apply one-hot encoding to the input
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes=27).float()
print(xenc)
print(xenc.shape)

In [32]:
# we can also visualize the one-hot encoding like this:
plt.imshow(xenc)

In [34]:
# check the data type of the one-hot encoding
xenc.dtype

## 9. The First Neuron

A neuron consists of a simple dot product $x\cdot W + b$. We don't use a **bias** $b$ here, so let's first initialize the **weights** $W$ randomly sampled from a normal distribution, i.e., most weights will be around 0:

In [ ]:
W = torch.randn((27, 1))
W

**TODO:** 9) Apply the matrix multiplication (denoted by `@` in PyTorch) to get the **logits**. Store the result in a variable called `logits`. Which shape will the result have and why? Check your predicted shape by printing the result! **(2 points)**

In [35]:
# YOUR CODE GOES HERE 

# Perform matrix multiplication to get logits
logits = xenc @ W

# Print the shape of logits to verify
print(logits.shape)

**ANSWER:** YOUR ANSWER GOES HERE

When we multiply xenc (of shape (num_bigrams, 27)) by W (of shape (27, 1)), the resulting shape will be (num_bigrams, 1). Each row in logits will represent the output (or logit) for one bigram in the dataset.

Since our num_bigrams size is 5, the result is of shape (5,1).

## 10. Creating 27 Neurons

Now we want 27 neurons instead of just one, because we eventually want to output the 27 probabilities for each character to be the next. So we want the weight matrix to be 27x27 instead of 27x1, and we will in parallel evaluate the 5 inputs on 27 neurons, so the output will be 5x27 (matrix multiplication of a 5x27 input with 27x27 weight matrix).

In [36]:
# randomly initialize 27 neurons' weights. Each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g)
print(W)

In [37]:
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
print(logits)

These numbers tell us the firing rate of each of the 27 neurons on the 5 inputs. For example, the result at row 3 and column 13 is giving us the firing rate of the 13th neuron looking at the third input:

In [38]:
(xenc @ W)[3,13]

 This is equivalent to a dot product between the third input and the 13th column of $W$, so using matrix multiplication we can very efficiently evaluate the dot product between lots of input examples in a batch and lots of neurons:

In [39]:
(xenc[3] * W[:,13]).sum() # element-wise multiplication and sum is equivalent to matrix multiplication, but less efficient

Here we can see the steps so far, including one-hot-encoding, weights and logit calculation for input 'e':

<img src="img/neuron1.jpg" width="500">

Note that in the image we only plot a single input 'e', whereas in the code we process 5 inputs in parallel.

We see that the logits are positive and negative. They can be interpreted as **log counts** (=logarithms of the counts of each bigram). We can elementwise exponentiate to transform them into numbers in $(0, \infty)$, where negative numbers go to $(0,1)$ and positive numbers go to $(1,\infty)$:

In [41]:
# Generate input values between -2 and 2
x = torch.linspace(-2, 2, 100)  # 

# Compute the exponential of the input values
y = torch.exp(x)

# Create the plot
plt.figure(figsize=(4, 3))
plt.plot(x, y, label='log(x)')
plt.title('Exponential of Positive and Negative Values')
plt.xlabel('Input Value')
plt.ylabel('Exponential')
plt.legend()
plt.grid(True)

**TODO:** 10a) Calculate the counts from the log counts by applying exponential function. Store the result in `counts` and print it. Normalize the counts like above, store the result in `probs` and print it. **(2 points)** 

In [42]:
# YOUR CODE GOES HERE 

# Calculate counts from log counts
counts = torch.exp(logits)  # Apply exponential function to each element in logits
print("Counts:\n", counts)

# Normalize counts to get probabilities
probs = counts / counts.sum(dim=1, keepdims=True)  # Sum across rows and divide
print("Probabilities:\n", probs)


Here are the visualized steps including exponentiation and normalization:

<img src="img/neuron2.jpg" width="700">

**TODO:** 10b) Do the last two maths operations in the cell above look familiar? How is this transformation called? **(1 point)**

**ANSWER:** YOUR ANSWER GOES HERE 

Yes, the last tow math operations combined are known as the softmax transformation.

Let's take a look at the resulting "probabilities" for the first character (of course they are meaningless because the network is still untrained, the weights are randomly initialized):

In [43]:
probs[0]

In [44]:
probs[0].shape

In [45]:
probs.shape

## 11. Summary (so far)

Let's summarize the code so far to get an overview of the steps:

In [46]:
xs # input to the network ('.emma')

In [47]:
ys # target for the network ('emma.')

In [48]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g)

In [49]:
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character

In [50]:
probs.shape # output probabilities of shape 5x27

In [51]:
# compute the negative log likelihood loss for the first 5 bigrams
nlls = torch.zeros(5)
for i in range(5):
    # i-th bigram:
    x = xs[i].item() # input character index
    y = ys[i].item() # label character index
    print('--------')
    print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indices {x},{y})')
    print('input to the neural net:', x)
    print('output probabilities from the neural net:', probs[i])
    print('label (actual next character):', y)
    p = probs[i, y]
    print('probability assigned by the net to the the correct character:', p.item())
    logp = torch.log(p)
    print('log likelihood:', logp.item())
    nll = -logp
    print('negative log likelihood:', nll.item())
    nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

Here is a visualization of the first feedforward pass of input 'e' through the neuron, including negative log likelihood loss calculation. You can see in the output above that the loss is roughly 4 (see 'bigram example 2' output).

<img src="img/neuron3.jpg" width="900">

We see from the output above that the average loss is 3.7, which is quite high. We can change the random seed for sampling $W$ for randomly changing the loss. But of course, we can do better: The loss calculation is only made up of differentiable operations (multiplication, addition, exponential, division...) We can minimize the loss by computing the gradients of the loss with respect to $W$. 

## 12. Optimization

To calculate the loss, we basically need to pluck out the predicted 
probabilities at the indices of the target characters. For example, for the first bigram,
the input character is '.' (index 0) and the target character is 'e' (index 5), and the predicted probability is stored in `probs[0,5]`.

In [52]:
# we need the following probabilities for calculating the loss:
probs[0,5], probs[1,13], probs[2,13], probs[3,1], probs[4,0] 

This is equivalent to:

In [53]:
probs[torch.arange(5), ys]

We can use this in the loss calculation at the end of the forward pass:

In [54]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True) # we need to compute gradients

In [55]:
# forward pass
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
loss = -probs[torch.arange(5), ys].log().mean() # average negative log likelihood loss

In [56]:
print(loss.item()) # print the loss

Note that the loss is the same as above. Finally we can do the backward pass:

In [57]:
# backward pass
W.grad = None # set to zero the gradient (None is a special value that PyTorch recognizes as "set to zero")
loss.backward() # compute the gradients

Something magical happened when `loss.backward()` was run: PyTorch keeps track of all the operations in the forward pass, under the hood it builds a full computational graph, so it knows all the dependencies and all the mathematical operations inside the neural network. When we calculate the loss and call a `.backward()` on it, PyTorch fills in the partial derivatives of all the intermediate steps. So now we can look at the gradient and see that it is not zero anymore:

In [58]:
W.grad

In [59]:
W.shape, W.grad.shape

Both `W` and `W.grad` are of shape 27x27, and each entry of `W.grad` is telling us the influence of that weight on the loss function. For example, since `W.grad[0,0]` is positive, slightly increasing `W[0,0]` will lead to a slightly bigger loss. Therefore we use the *negative* gradient for updating the weights:

In [60]:
W.data += -0.1 * W.grad # update the weights with learning rate 0.1

Let's check that the loss has really decreased using this one **gradient descent** step:

In [61]:
# forward pass
xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
loss = -probs[torch.arange(5), ys].log().mean()
print(loss.item())

## 13. Optimizing over the Whole Dataset

Now let's apply gradient descent to the whole dataset:

In [62]:
# create the dataset
xs, ys = [], []
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement() # number of elements in the tensor xs (5 in '.emma.', here more)
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [63]:
# gradient descent
for k in range(500):
  
    # forward pass
    xenc = F.one_hot(xs, num_classes=27).float() # input to the network: one-hot encoding
    logits = xenc @ W # predict log-counts - one-hot input will pluck the correct row from W, like in the previous bigram model
    counts = logits.exp() # counts, equivalent to N
    probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
    loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
    print(loss.item())
    
    # backward pass
    W.grad = None # set to zero the gradient
    loss.backward()
    
    # update
    W.data += -50 * W.grad # even a large learning rate is fine here, because the bigram model is very simple

We see that the loss is converging roughly to 2.48, which is similar to the loss we achieved all the way up in the original bigram model without a neural network (2.45). Back then, we achieved this loss by counting, now by optimizing weights. That makes sense because fundamentally we're not using any additional information, we're still just taking in the previous character and trying to predict the next one, but instead of doing it explicitly by counting and normalizing we are doing it with gradient-based learning. The explicit approach happens to very well optimize the loss function without any need for a gradient based optimization because the setup for bigram language models is so straightforward, we can just afford to estimate those probabilities directly and maintain them in a table. But the gradient-based approach is significantly more flexible, so we've actually gained a lot because we can expand this approach and complexify the neural net.

**TODO (optional):** Remember the model smoothing used in the first bigram model by adding a number to the counts to avoid zero probabilities. Can you think of a smoothing equivalent in gradient descent based bigram model?

**HINT:** Think of the weight matrix $W$ and its influence on the result!

**ANSWER:** YOUR ANSWER GOES HERE 

The code above already includes a smoothing:

`loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()`

While the last part `+ 0.01*(W**2).mean()` is a regularization term which is proportional to `W**2` which penalizes large values in `W` and encourages a smoother and less extreme distribution of weights.

Let's see how the weights and the probabilities have changed after training:

In [64]:
print(W)

In [65]:
# compute the negative log likelihood loss for the first 5 bigrams
nlls = torch.zeros(5)
for i in range(5):
    # i-th bigram:
    x = xs[i].item() # input character index
    y = ys[i].item() # label character index
    print('--------')
    print(f'bigram example {i+1}: {itos[x]}{itos[y]} (indices {x},{y})')
    print('input to the neural net:', x)
    print('output probabilities from the neural net:', probs[i])
    print('label (actual next character):', y)
    p = probs[i, y]
    print('probability assigned by the net to the the correct character:', p.item())
    logp = torch.log(p)
    print('log likelihood:', logp.item())
    nll = -logp
    print('negative log likelihood:', nll.item())
    nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

**TODO:** 13) Here is the final visualization after training. Can you interpret the weights, have they improved? Can you compare the trained weights to the weights from our first approach using simple counting instead of a neuron? **(2 points)** 


<img src="img/neuron4.jpg" width="900">


**ANSWER:** YOUR ANSWER GOES HERE 

The weights from the first approach using simple counting can be thought of as counts of occurrences normalized by the total occurrences of the first character in each bigram. However, the weights of the trained neuron are an improvement in the following way:

The weights do not just reflect direct counts but learned patterns that capture dependencies between characters more effectively than raw counts. Means, the weight is higher if a dependency occurred often. Also the trained neuron reflects a smoothing effect: even if a certain bigram wasn't seen during the training, the model still assign a nonzero probability to it. 

## 14. Sampling from the Neural Net

In [66]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(20):
  
    out = []
    ix = 0
    while True:
        
        # ----------
        # BEFORE:
        #p = P[ix] # bigram probabilities by counting
        # ----------
        # NOW:
        xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
        logits = xenc @ W # predict log-counts
        counts = logits.exp() # counts, comparable to N
        p = counts / counts.sum(1, keepdims=True) # probabilities for next character
        # ----------
        
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    print(''.join(out))

We get nearly identical samples as in the first model!

## 15. Conclusion

To summarize, we introduced the bigram character level language model, we saw how we can train the model, how we can sample from the model and how we can evaluate the quality of the model using the negative log likelihood loss. We trained the model in two completely different ways that actually get the same result and the same model: First, we just counted the frequency of all the bigrams and normalized. Second, we used the negative log likelihood loss as a guide to optimizing the counts matrix so that the loss was minimized in a gradient-based framework. Both approaches gave the same result, but the gradient-based framework is much more flexible and we can extend it to more complex settings.